In [ ]:
import os
os.makedirs("outputs", exist_ok=True)
print("✅ Feature engineering ready")


Cloning into 'technosignature-pipeline-v2'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 89 (delta 31), reused 23 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (89/89), 32.86 KiB | 3.65 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [ ]:
import pandas as pd
from astropy.coordinates import SkyCoord, match_coordinates_sky
import astropy.units as u

# Load master & all flag tables
master = pd.read_csv("inputs/confirmed_planets.csv")
wise   = pd.read_csv("inputs/wise_photometry_icrs.csv")
ps     = pd.read_csv("inputs/panstarrs.csv")
sd     = pd.read_csv("inputs/sdss.csv")
bl     = pd.read_csv("inputs/bl_filtered_icrs.csv")
tess   = pd.read_csv("inputs/tess_flags.csv")
gaia   = pd.read_csv("inputs/gaia_flags.csv")
spec   = pd.read_csv("inputs/spec_flags.csv")

coords_pl = SkyCoord(master["ra"]*u.deg, master["dec"]*u.deg)

# IR flag (5″ cone)
coords_w = SkyCoord(wise["ra_icrs"]*u.deg, wise["dec_icrs"]*u.deg)
_,d2d,_ = match_coordinates_sky(coords_pl, coords_w)
master["ir_flag"] = d2d < 5*u.arcsec

# Pan‑STARRS flag by name
master = master.merge(ps[["pl_name","ps_flag"]], on="pl_name", how="left")

# SDSS flag by name
master = master.merge(sd[["pl_name","sdss_flag"]], on="pl_name", how="left")

# BL, TESS, Gaia, Spec as before
coords_b = SkyCoord(bl["ra_icrs"]*u.deg, bl["dec_icrs"]*u.deg)
_,d2d,_ = match_coordinates_sky(coords_pl, coords_b)
master["radio_flag"] = d2d < 1*u.arcsec

master = master.merge(tess[["tic_id","tess_flag"]], on="tic_id", how="left")
master = master.merge(gaia[["pl_name","gaia_flag"]], on="pl_name", how="left")
master["spec_flag"] = False  # or your mapping

# Fill NaNs
for col in ["ps_flag","sdss_flag","tess_flag","gaia_flag"]:
    master[col] = master[col].fillna(False)

# Save
master.to_csv("outputs/feature_matrix.csv", index=False)
print("✅ feature_matrix.csv with PS & SDSS flags saved")
